### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной и, где это возможно, не использовать циклы

Необходимо реализовать класс KnnBruteClassifier, с реализацией прототипа, представленного ниже.

Должна быть реализована поддержка метрики расстояния L2 (параметр metric) и параметр weights типа 'uniform' и 'distance'.

В качестве входного файла необходимо использовать файл "knn_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn.ipynb - содержит класс, реализующий ваш алгоритм
* results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
import numpy as np

In [ ]:
class KnnBruteClassifier(object):
    '''Классификатор реализует взвешенное голосование по ближайшим соседям.
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    '''
    def __init__(self, n_neighbors=1, weights='uniform', metric="l2"):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        if metric == "l2": self.dist = lambda a,b: np.sqrt( np.sum((a-b)**2))
        pass

    def fit(self, x, y):
        '''Обучение модели.
        Парметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        '''
        self.x = x
        self.y = y
        self.classes = np.unique(y)
        return self

    def predict(self, x):
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
        y = np.zeros(len(x))
        neigh_dist, neigh_indarray = self.kneighbors(x, self.n_neighbors)

        classind = lambda i: np.where(self.classes == self.y[int(i)])

        neigh_class_ind = np.array([[classind(i) for i in v] for v in neigh_indarray])

        if self.weights == 'uniform':
          for idx, v in enumerate(neigh_class_ind):
            amounts = np.zeros(len(self.classes))
            for i in v:
              amounts[i]+=1
            ans = self.classes[np.argmax(amounts)]
            y[idx] = ans
        elif self.weights == 'distance':
          for idx, v in enumerate(neigh_class_ind):
            amounts = np.zeros(len(self.classes))
            for k, i in enumerate(v):
              amounts[i] += 1 / neigh_dist[idx,k]
            ans = self.classes[np.argmax(amounts)]
            y[idx] = ans
        return y


    def predict_proba(self, X):
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности
        объекта к каждому классу
        """
        p = np.zeros((len(X), len(self.classes)))
        for i, cl in enumerate(self.predict(X)):
         clidx = np.where(self.classes == cl)
         p[i,clidx] += 1

        return p


    def kneighbors(self, x, n_neighbors):
        """Возвращает n_neighbors ближайших соседей для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """
        neigh_dist = np.zeros((len(x), n_neighbors))
        neigh_indarray = np.zeros((len(x), n_neighbors))

        for i,v in enumerate(x):
          distance = np.array([self.dist(v,a) for a in self.x])#len = len(self.x)
          neighbors = np.argsort(distance)[:n_neighbors]
          neighbors_distances = distance[neighbors]

          neigh_indarray[i] = neighbors
          neigh_dist[i] = neighbors_distances

        return neigh_dist, neigh_indarray

In [ ]:
def load_file(filename):
    """
    TODO: Необходимо загрузить файл задания и вернуть словарь с ключами "X_train", "X_test", "y_train"
    """
    data = np.load(filename, allow_pickle=True)
    return data.item()
    #return np.load(filename, allow_pickle=True)
    #return {"X_train": X_train , "X_test": X_test , "y_train": y_train }

In [ ]:
input_filename = "/content/knn_data_039.npy" #TODO задать путь к входному файлу
data_dict = load_file(input_filename)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model2 = KNeighborsClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
model2.fit(data_dict["X_train"], data_dict["y_train"])
print( model.kneighbors(data_dict["X_test"],n_neighbors=5)[1] == model2.kneighbors(data_dict["X_test"],n_neighbors=5 )[1])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])

[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 ...
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


In [ ]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])

In [ ]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])

In [ ]:
output_filename = "results.npy"
result_dict = {
    "input_filename": input_filename,
    "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
    "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
    "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
}
np.save(output_filename, result_dict)